# 1번 영상

## Event indexing
매개변수를 인덱싱한다는 것은, 빠른 검색을 위해 색인을 만들어 놓는다는 의미와 같음. 매개변수 3개까지는 인덱싱이 가능함

예를 들어 Transfer(address indexed _from, address indexed _to, uint256 _value) 이렇게 인덱스가 있다면, 이 경우 전체가 아닌 송신주소, 수신 주소로 국한지어 검색 가능함
> contract.Event({from:"주소"}, {fomr Block: , toBlock : 'latest'});

이렇게 송신측의 주소를 잡아놓게 되면, 이 주소에서 보낸경우만 첫번째 블록부터 가장 마지막에 생성된 블록까지 인덱스를 잡아서 검색해줌

## Event Overloading
컨트랙에 이벤트를 여러개 가질 수 있는지의 문제. 

OrderEvent.sol파일을 만들어서 해보자.
리믹스에서 Deploy를 한 후 Deployde Contracts에서, order에 값을 넣어주자.
가장 맨 첫번째는 String이므로 "0x1234"라고 쓰고, ","뒤로 쭉 이어 나가자.
0x를 붙이는 이유는 16진수 hex표기를 위함이고, 
1234는 2bytes니 그냥 임의로 넣어준 것.
그리고 정수 값을 주면 된다. 


order를 하고 나면 아무 값도 뜨지 않는다.
디버그 창을 보면 vm 오류가 나 있음을 확인할 수 있다. 
오류 메세지는 하다가 중단된 오류란다. 이를 디버그해보자. 
디버그 버튼을 누르면 125번째 opcode 부터 실행되고 있는 것이 보인다. 


아래 4개의 화살표 중 3번째(아래로 꼬꾸라진)화살표를 눌러보자. 
그러면 127번째 Timestamp로 이동한다. 
그리고 해당하는 opcode에서 gas가 2만큼 일어나는 것이 보인다. 


그 아래 Solidity Locals를 확인해보면, 
우리가 조금 전 입력한 로컬 변수의 값이 나오고 있고, 
state 값, Stack은 로컬함수에서 실행될 때 스택은 명령문을 실행하는 방식인 것이다.
아래 Memory 변수 상태, Storage 변수 상태에 대해서도 볼 수 있다. 

오류는 어디서 난 것일까? opcode를 쭉 내리다보면 148 REVERT가 있다.
오류는 REVERT에서 난 것이다. 컴파일 오류는 없는데, 런타임 오류가 난 것이다.
런타임 오류가 발생한 것. 이는 fuction order에서 오류가 난 것인데,
메시지 밸류를 잡아주지 않았기 때문이다.

이를 해결하려면 디폴로이를 할 때 value에 30을 넣어주면 된다.
##### 3*10의 개념!!! 그 후 order에는 똑같이 "0x1234",3 넣어주기

그리고 로그창을 보면 성공한 것을 확인할 수 있다! (그린 버튼이 뜸)
이는 order 트랜잭션이 성공적으로 일어났다는 것이다. 

쭈우욱 내려서 로그를 확인하면
> 	[
	{
		"from": "0x566870f3b037d5523c00e1619b33369c2406839e",
		"topic": "0xb9e815be6bf0b588396af8fa0244c9f92e26ad4dbf3694c09d90b0cc9f2be8e5",
		"event": "OrderLog",
		"args": {
			"0": "Ordered",
			"length": 1
		}
	},
	{
		"from": "0x566870f3b037d5523c00e1619b33369c2406839e",
		"topic": "0x2fb1e4e17675a393b0a43253759334f7e61806d1ac2690f0432362ed1c4c1611",
		"event": "OrderLog",
		"args": {
			"0": "1591863238",
			"timestamp": "1591863238",
			"length": 1
		}
	},
	{
		"from": "0x566870f3b037d5523c00e1619b33369c2406839e",
		"topic": "0x1f7c3420c50f5b8875456d867e023a6d440ef2c1706d52ba38f5c6f553b53d50",
		"event": "OrderLog",
		"args": {
			"0": "0xe3C6f6468dA9a410FC353ec6F7E9833f61681535",
			"1": "0x1234",
			"2": "30",
			"_from": "0xe3C6f6468dA9a410FC353ec6F7E9833f61681535",
			"_itemId": "0x1234",
			"_value": "30",
			"length": 3
		}
	}
]

이와같이 내가 준 값대로 잘 나오고 있음을 확인할 수 있다.

## 1단계 : 컨트랙 개발

In [10]:
%%writefile src/OrderEvent.sol

pragma solidity ^0.6.0;

contract OrderEvent {
    uint unitPrice = 10; // 가격
    event OrderLog(string);
    event OrderLog(bytes2 _itemId, uint _value); // 주문하는 품목의 id
    event OrderLog(uint256 timestamp);
    event OrderLog(address indexed _from, bytes2 _itemId, uint indexed _value);

    function order(bytes2 _itemId, uint quantity) public payable { // item ID, 주문량을 매개변수로 받음. 지급이 일어나므로 payable
        uint256 orderTime = now; // 주문시간
        uint256 orderAmount = quantity * unitPrice; // 금액 구하기
        require(msg.value == orderAmount); // 입금 금액이 orderAmount와 같도록
        emit OrderLog("Ordered"); // String에 대한 것
        emit OrderLog(orderTime); // TimeStamp
        emit OrderLog(msg.sender, _itemId, msg.value);
    }
}

Overwriting src/OrderEvent.sol


## 2단계 : 컴파일

In [11]:
!solc --abi --bin --gas src/OrderEvent.sol


======= src/OrderEvent.sol:OrderEvent =======
Gas estimation:
construction:
   20159 + 102600 = 122759
external:
   order(bytes2,uint256):	5601
Binary:
6080604052600a60005534801561001557600080fd5b50610201806100256000396000f3fe60806040526004361061001e5760003560e01c806332af7f7314610023575b600080fd5b61007a6004803603604081101561003957600080fd5b8101908080357dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff191690602001909291908035906020019092919050505061007c565b005b6000429050600080548302905080341461009557600080fd5b7fb9e815be6bf0b588396af8fa0244c9f92e26ad4dbf3694c09d90b0cc9f2be8e56040518080602001828103825260078152602001807f4f7264657265640000000000000000000000000000000000000000000000000081525060200191505060405180910390a17f2fb1e4e17675a393b0a43253759334f7e61806d1ac2690f0432362ed1c4c1611826040518082815260200191505060405180910390a1343373ffffffffffffffffffffffffffffffffffffffff167f1f7c3420c50f5b8875456d867e023a6d440ef2c1706d52ba38f5c6f553b53d508660405180827dfffffffffffffffffffffffffff

## 3단계 : 배포
abi와 bin을 바꿔주세용

In [12]:
%%writefile src/OrderEventDeploy.js

var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var _abiArray = [{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"","type":"string"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"indexed":false,"internalType":"uint256","name":"_value","type":"uint256"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"timestamp","type":"uint256"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"_from","type":"address"},{"indexed":false,"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"indexed":true,"internalType":"uint256","name":"_value","type":"uint256"}],"name":"OrderLog","type":"event"},{"inputs":[{"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"internalType":"uint256","name":"quantity","type":"uint256"}],"name":"order","outputs":[],"stateMutability":"payable","type":"function"}];
//var _abiArray = JSON.parse(_abiStr);
var _bin="0x" + "6080604052600a60005534801561001557600080fd5b50610201806100256000396000f3fe60806040526004361061001e5760003560e01c806332af7f7314610023575b600080fd5b61007a6004803603604081101561003957600080fd5b8101908080357dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff191690602001909291908035906020019092919050505061007c565b005b6000429050600080548302905080341461009557600080fd5b7fb9e815be6bf0b588396af8fa0244c9f92e26ad4dbf3694c09d90b0cc9f2be8e56040518080602001828103825260078152602001807f4f7264657265640000000000000000000000000000000000000000000000000081525060200191505060405180910390a17f2fb1e4e17675a393b0a43253759334f7e61806d1ac2690f0432362ed1c4c1611826040518082815260200191505060405180910390a1343373ffffffffffffffffffffffffffffffffffffffff167f1f7c3420c50f5b8875456d867e023a6d440ef2c1706d52ba38f5c6f553b53d508660405180827dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff19167dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916815260200191505060405180910390a35050505056fea26469706673582212207229691c42a8c3b0a93b507e5c0295755bc82ac4f4745461a9a2eb8ceb91045b64736f6c63430006030033";

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() { //async 이용합니댜
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 300000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance) {
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}

deploy()

Overwriting src/OrderEventDeploy.js


In [13]:
!node src/OrderEventDeploy.js

Deploying the contract from 0x5459faa8B1d456D9b30e9eD324f65d0a7390223a
hash: 0x58829ae7172cfec87e84d12c4454a084b5cbfc7f2e9b8ff1f11786afd5c5d17a
---> The contract deployed to: 0x7A319C712a37b80212a20375dE4eDDFA8ADAAc43


## 4단계 : 사용

In [8]:
%%writefile src/OrderEventUse.js

var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    var web3 = new Web3(new Web3.providers.WebsocketProvider("http://127.0.0.1:8345"));
}
var _abiArray = [{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"","type":"string"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"indexed":false,"internalType":"uint256","name":"_value","type":"uint256"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"timestamp","type":"uint256"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"_from","type":"address"},{"indexed":false,"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"indexed":true,"internalType":"uint256","name":"_value","type":"uint256"}],"name":"OrderLog","type":"event"},{"inputs":[{"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"internalType":"uint256","name":"quantity","type":"uint256"}],"name":"order","outputs":[],"stateMutability":"payable","type":"function"}];
//var _abiArray=JSON.parse(_abiStr);

async function doIt() {
    var _order = new web3.eth.Contract(_abiArray, '0xDFEaf2894F334cE23C7fb78a77eA6f790dB32aae');
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    var event = _order.events.OrderLog({
            //filter: {_from: accounts[0], _value: [20,50]},
            filter: {_from: accounts[0], _value: 30}, //주소를 넣음 
            fromBlock: 'latest', toBlock: 'pending'
        }, function (error, result) { //callback 함수, error first 문법, result가 object로 출력되므로 JSON.stringify 이용
        if (!error) { //매개변수 없이 불러주기. 
            //console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
            console.log("Event fired: " + JSON.stringify(result.returnValues));
            //process.exit(1); //대기를 끊고 싶을 때 강제종료 시키는 문구
        }
    });
    var value;
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    // this will fire an event
    my = await _order.methods.order("0x1234", 3) //여기서 불러주는 중, 리믹스에서 쓴 것처럼 쓴다
        .send({from: accounts[0], gas: 100000, value:30})
        //.then(function(my) {console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    // this will fire another event
    my = await _order.methods.order("0x1234", 4).send({from: accounts[0], gas: 100000, value:40});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    // this will NOT fire another event
    my = await _order.methods.order("0x1234", 10).send({from: accounts[0], gas: 100000, value:100});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    process.exit(1); 
}

doIt()

Overwriting src/OrderEventUse.js


In [9]:
!node src/OrderEventUse.js

Account: 0x4e4c618aCfE7D06279914b87D0777F20F7dB0833
Balance before: 99994702439999999830
Event fired: {"0":"0x4e4c618aCfE7D06279914b87D0777F20F7dB0833","1":"0x1234","2":"30","_from":"0x4e4c618aCfE7D06279914b87D0777F20F7dB0833","_itemId":"0x1234","_value":"30"}
---> MyFunction called undefined
---> MyFunction called undefined
---> MyFunction called undefined
Balance after: 99993085019999999660
Balance diff: 1617420000002048


함수가 불리어졌다는 사실은 위의 ---> MyFunction called 부분을 통해 알 수 있다. 

# 파일에 로그쓰고 로그 내용을 봐보쟈

In [61]:
%%writefile src/OrderEventUse1.js

var fs=require('fs');
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    var web3 = new Web3(new Web3.providers.WebsocketProvider("http://127.0.0.1:8345"));
}


var _abiArray=[{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"","type":"string"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"indexed":false,"internalType":"uint256","name":"_value","type":"uint256"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"timestamp","type":"uint256"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"_from","type":"address"},{"indexed":false,"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"indexed":true,"internalType":"uint256","name":"_value","type":"uint256"}],"name":"OrderLog","type":"event"},{"inputs":[{"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"internalType":"uint256","name":"quantity","type":"uint256"}],"name":"order","outputs":[],"stateMutability":"payable","type":"function"}];

async function doIt() {
    var _order = new web3.eth.Contract(_abiArray, '0x7A319C712a37b80212a20375dE4eDDFA8ADAAc43');
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    var event = _order.events.OrderLog({//7
            filter: {_from: accounts[0], _value: 30}, //주소를 넣음 6
            fromBlock: 'latest', toBlock: 'pending'
        }, function (error, result) { //7,5
        if (!error) { 
            log = JSON.stringify(result.returnValues);
            console.log("Event fired: " + log);
            {//3
                fs.appendFile("src/OrderEventList.txt", log, "utf-8", function(e){//2
                if(!e){//1
                    console.log(">> Writing to file");
                }//1
            }//2
        )}//3
        }//4
    });//5
    
    var value;
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    my = await _order.methods.order("0x1234", 3) //여기서 불러주는 중, 리믹스에서 쓴 것처럼 쓴다
        .send({from: accounts[0], gas: 100000, value:30})
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    process.exit(1);   
    }

doIt()

Overwriting src/OrderEventUse1.js


In [58]:
!node src/OrderEventUse1.js

Account: 0x5459faa8B1d456D9b30e9eD324f65d0a7390223a
Balance before: 99996319860000000000
Event fired: {"0":"0x5459faa8B1d456D9b30e9eD324f65d0a7390223a","1":"0x1234","2":"30","_from":"0x5459faa8B1d456D9b30e9eD324f65d0a7390223a","_itemId":"0x1234","_value":"30"}
>> Writing to file
---> MyFunction called undefined
Balance after: 99995780719999999970
Balance diff: 539140000006144


In [59]:
!type C:\Users\15Z970-G.AA5BK\Code\jsr\src\OrderEventList.txt

{"0":"0x5459faa8B1d456D9b30e9eD324f65d0a7390223a","1":"0x1234","2":"30","_from":"0x5459faa8B1d456D9b30e9eD324f65d0a7390223a","_itemId":"0x1234","_value":"30"}


# 2번 영상
왜인지는 모르겠는데 Multiply7Event에 대한 것...
이거는 week12_lab이랑 같음요 
과제 풀이 후 다음꺼 봐봅시다

## fallback 함수
이는 사용자가 호출하지 않는다. 때문에 함수는 이름도, 인자도, 반환 값도 없는 함수이다. 
##### 컨트랙마다 단 1개의 fallback함수만 존재할 수 있다.
외부에서 호출할 수 있도록 extern이라는 가시성을 붙여 넣는다.


fallback 함수가 없으면 당연히 예외가 발생한다.
fallback 함수가 있으면 예외가 발생하지 않는다고 보면 된다.


### fallback 함수를 사용하는 2가지 경우

- 사용자가 호출한 함수가 존재하지 않을 경우 호출이 됨
  ex. callA()를 호출해야 하는데 callB()를 호출했다? 그러면 존재하지 않는 callB()대신 fallback()을 실행함
  
- 또한 송금을 했는데 수신 측에서 이를 받는 함수가 없을 때
  송금이 거절되지만 fallback()이 호출된다. 이 경우 fallback() payable로 선언해야 함
  
  
## receive 함수
이는 송금을 하지만 calldata가 없는 경우 호출되고, receive() external payable로 사용해야 함.
receive함수가 없으면 대신 fallback()함수가 호출됨

### receive 함수가 실행되는 2가지 경우
- receive() external payable은 calldata가 없고, value 필드에 값이 있는 경우
- fallback() external payable은 해당하는 함수가 없는 경우. 송금이 있는 경우에는 payable은 선택적으로, 즉 송금 금액이 있는 경우에는 payable을 적어줌

우선 리믹스로 만들어보자. 컴파일 후, Deploy를 하고, callA를 눌러보자. 

그렇다면 fallback함수는 어떻게 호출하지?
입력창이 없는데? 그럴때에는 아래 있는 'Low Level Interactions' 창을 보면 CALLDATA라고 있다.
여기에 16진수를 넣어보자. 주의할 점은, 2글자가 1byte이므로 짝수개로 넣어야 한다.

넣고나서 log창을 확인해보면...
> [
	{
		"from": "0x4618ef51121314627889b6616688e9247e14b345",
		"topic": "0x968f0302429ff0e5bd56a45ce3ba1f4fa79f4b822857e438616435f00c3b59fd",
		"event": "PrintLog",
		"args": {
			"0": "fallback called",
			"length": 1
		}
	}
]

이렇게 나온다. 사용자가 직접 fallback을 호출할 수 없으므로 컨트랙 간에 호출되는 것이다.
web3로 강제로 출력하려면 callDATA를 수정해야한다. 

## 1단계 : 컨트랙 개발

In [18]:
%%writefile src/FallbackTest.sol

pragma solidity ^0.6.0;
contract FallbackTest {
    event PrintLog(string);
    function callA () pure public returns(string memory){ // String은 reference함수니까 memory를 적어주어야 함
        return "doing callA"; // 문자열 반환
    }
    fallback () external { // callB호출 시 나오는 fallback함수
        emit PrintLog("fallback called");
    }
}

Overwriting src/FallbackTest.sol


## 2단계 : 컴파일

In [37]:
!solc src/FallbackTest.sol --combined-json abi,bin > src/FallbackTest.json

In [41]:
!type C:\Users\15Z970-G.AA5BK\Code\jsr\src\FallbackTest.json

{"contracts":{"src/FallbackTest.sol:FallbackTest":{"abi":"[{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"internalType\":\"string\",\"name\":\"\",\"type\":\"string\"}],\"name\":\"PrintLog\",\"type\":\"event\"},{\"stateMutability\":\"nonpayable\",\"type\":\"fallback\"},{\"inputs\":[],\"name\":\"callA\",\"outputs\":[{\"internalType\":\"string\",\"name\":\"\",\"type\":\"string\"}],\"stateMutability\":\"pure\",\"type\":\"function\"}]","bin":"608060405234801561001057600080fd5b50610190806100206000396000f3fe608060405234801561001057600080fd5b506004361061002f5760003560e01c8063e7f09e051461009a57610030565b5b7f968f0302429ff0e5bd56a45ce3ba1f4fa79f4b822857e438616435f00c3b59fd60405180806020018281038252600f8152602001807f66616c6c6261636b2063616c6c6564000000000000000000000000000000000081525060200191505060405180910390a1005b6100a261011d565b6040518080602001828103825283818151815260200191508051906020019080838360005b838110156100e25780820151818401526020810190506100c7565b50505050905090810190601f168015610

## 3단계 : 배포

In [72]:
%%writefile src/FallbackTestDeployFromFile.js

var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/FallbackTest.json"); //파일 읽어오기
var _json=JSON.parse(_str) //string을 JSON형식으로 바꾸어줌
var _abiArray=JSON.parse(_json.contracts["src/FallbackTest.sol:FallbackTest"].abi);
//console.log(_abiArray);
//var _bin=_json.contracts.sHello2.bin;
var _bin="0x"+_json.contracts["src/FallbackTest.sol:FallbackTest"].bin;
//console.log(_bin);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/FallbackTestDeployFromFile.js


In [73]:
!node src/FallbackTestDeployFromFile.js

Deploying the contract from 0xC44e483669dD1609bA680B47bA3aE59ccA5404b4
hash: 0x2b892002666e0a8a479956a28a6b3a1dc835497a2994ed4a58a33297728faf4e
---> The contract deployed to: 0xA52c12CEf48428Bd34B4949C4bd2cd1cC3ed872b


## 4단계 : 사용

In [74]:
%%writefile src/FallbackTestUseFromFile.js

var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/FallbackTest.json");
var _json=JSON.parse(_str)
var _abiArray=JSON.parse(_json.contracts["src/FallbackTest.sol:FallbackTest"].abi);
var _instance = new web3.eth.Contract(_abiArray, "0xA52c12CEf48428Bd34B4949C4bd2cd1cC3ed872b");
var event = _instance.events.PrintLog(function (error, result) {
    if (!error) {
        console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
    } //이벤트 발생시키기
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    _instance.methods.callA().call().then(console.log);  //null
    //call without calling any method //바로 아래에서 callB를 호출하고 있음. 이로인해 오류가 날 것. 
    //await _instance.methods.callB().send({from:accounts[0], to: 0x08c6FB3fFDfdf5d46db22A7e9e8Ac8DdC930bbC9});
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()

Overwriting src/FallbackTestUseFromFile.js


### 존재하지 않는 function selector
존재하지 않는 함수를 호출하면 fallback함수가 호출됨. 
클라이언트에서 web3를 사용하면, 존재하지 않는 함수를 호출할 수 있는 방법은 없음.

가능한 방법은 web3.eth.sendTransaction()의 data, 즉 msg.data(전역변수)를 수정하면 됨 

앞서 설명한 #### function selector (호출할 함수시그니처의 sha3 해시값으로 만들고, 앞 4바이트)가 존재하지 않도록 수정해야 함.

> <address>.call(bytes4(bytes32(sha3("baz(uint32,bool)")))
   
아쉬운대로 다음과 같이 함수명을 생략하여 강제적으로 존재하지 않는 함수를 호출.
    
즉 컨트랙 주소로 그냥 트랜잭션을 전송하여 없는 함수가 호출된 것처럼 한다.

> web3.eth.sendTransaction({from:web3.eth.accounts[0], to:<<contract address>>});

In [75]:
!node src/FallbackTestUseFromFile.js

Account: 0xC44e483669dD1609bA680B47bA3aE59ccA5404b4
Balance before: 99994425720000000000
doing callA
Balance after: 99994425720000000000
Balance diff: 0


## payable fallback
앞서 존재하지 않는 함수 호출 시, fallback함수가 호출되었다. 
이번에는 송금하는 경우에, 함수가 존재하지 않으면 payable fallback함수가 호출됨 

여기서는 컨트랙을 두개를 만들고 있음. 
- fallback함수를 가지고 있는 컨트랙
- 이를 호출하는 컨트랙을 만들어야함

이 컨트랙을 서로 연관지어야 함.new명령어를 이용해서 객체를 만들거나, 
이미 배포된 컨트랙을 사용하려고 하면 그 주소를 할당해서 사용하고 있음

## 1단계 : 컨트랙 개발

리믹스에서 Multiply7 컨트랙과 Math 컨트랙을 만들어보자.

Multiply7은 fallback함수를 가질 거고, Math에서는 fallback함수를 호출하게 될 것.

솔리디티 파일 이름은 굳이 컨트랙 이름과 같게 할 필요가 없으므로 "MathMultiply.sol"이라고 만들어준다.


### 리믹스에서 코드를 작성한 후, Deploy해보자.
- 컴파일을 하고나면 CONTRACT에 2개가 뜨는 것을 볼 수 있다.
- 우선 Multiply7부터 Deploy 해보자.
- multiply에 8을 넣어서 클릭하면 56이 나오고, getAddress를 누르면 주소가 나온다.
- 나온 주소를 복사 해놓자.


- 이번에는 Math를 Deploy한다.
- getAddressOf로 주소를 출력해보자. 이는 multiply에서 나온 주소가 아닌, new 명령어로 가져온 주소가 출력이 된다. 
- 이후 getBalanceOfM7, getBalanceOfThis의 잔고를 눌러보면 당연히 0이 나온다.
- 그리고 multiply도 눌러보면 56이 제대로 나오고 있다.

- deposit을 해야 송금이 된다. deposit에 123을 넣고 value도 123을 넣고, deposit 버튼을 눌러주자.
- 그리고 M7의 잔고를 확인하면 0이, This의 잔고를 확인하면 123이 나온다.


- 이번에는 send11M7을 누르고 M7의 잔고를 확인하면 11이 나온다.
- 반면 This의 잔고는 11이 줄어들어 있음을 확인할 수 있다. 

#### 잔고는 왜 줄었을까?
send11M7을 눌렀을 때 multiply7의 payable fallback함수가 호출이 된 것임.


- 이번에는 주소를 변경을 해서 실행해보자.
- setM7에 아까 복사한 주소를 붙여넣는다. 그리고 setM7을 누른다.
- getAddress로 주소를 보자. 방금 복붙한 주소로 바뀌어있다.
- M7잔고는 다시 만들었으니 잔고가 0이 된다.
- This의 잔고는 112로 동일하다.


- 그러면 새로 바뀐 주소에 fallback함수가 적용되는지 해보자.
- send11M7을 누른다. 그리고 This의 잔고를 보면 11만큼 또 줄어있다.
- M7의 잔고는 11이 늘어나 있다. 

#### 두가지 방법으로 두 컨트랙을 연결시켜 보았다.
- new를 사용하는 경우는 같은 파일에 컨트랙이 존재할 경우에 사용한다.
- 주소는 이미 배포 되어 있는 경우에, 주소를 복사해서 객체를 만들어 사용하는 것이다. 이 경우에도 send함수를 통해서 payable fallback()이 호출이 되고, 전송이 되는 것을 확인할 수 있었다.

In [79]:
%%writefile src/MathMultiply7.sol

pragma solidity ^0.6.0;
contract Multiply7 {
   //event Print(uint);
   receive() external payable {} // receiver()는 fallback함수가 없을 때 호출되며, 송금이 될 수 있는 함수로 선언한다. 
   fallback() external payable {}  // body에 이벤트를 발생하는 함수를 넣어도 좋지만 안넣겠다!
   function multiply(uint input) pure public returns (uint) {
      //emit Print(input * 7);
      return input * 7;
   }
   function getAddress() view public returns(address) { // 주소 확인 함수, 컨트랙도 주소가 있고 외부 사용자 계정도 주소가 있다. 
       return address(this); // 주소 반환
   }
}

contract Math {
    Multiply7 m7=new Multiply7(); // 두 컨트랙을 연결시키기 위해 컨트랙의 객체를 만든다.
    // 같은파일이니 new명령어를 이용하고, 다른 파일이면 주소를 이용해야함
    function deposit(uint amount) payable public { 
        require(msg.value==amount); // 입력 값이 msg.value와 동일할 때에만 입금이 되도록함
    }
    function setM7(address payable _addr) public { // 주소를 이용해서 setting. 입력이 address
        m7 = Multiply7(_addr); // 주소를 넣어서 객체 생성
    }
    function multiply() view public returns(uint) { // Multiply7에 있는 함수를 호출해서 결과를 보여주는 것
        uint res=m7.multiply(8); // abi없이 함수를 호출해서 쓰는 것, 결과값을 받아놓는다.
        //this.send11(); It does not send value.
        return res;
    }
    function send11M7() public payable {
        // multiply7의 fallback함수, 돈을 송금하는 함수로, 송금 시 M7으로 입금이 됨.
        //m7.multiply.value(11)(9);
        //m7.multiply(9);
        address(m7).transfer(11); // 11wei만 address로 형변환 해준 m7을 호출함
    }
    function getBalanceOfThis() public view returns(uint) { // 잔고구하기
        return address(this).balance;
    }
    function getBalanceOfM7() public view returns(uint) { //multiply7 컨트랙의 객체인 m7의 잔고를 구하는 함수
        return address(m7).balance;
    }
    function getAddressOfM7() view public returns(address) { // 주소구하기
        return address(m7); //형변환 후 !! 봐야함
   }    
}

Writing src/MathMultiply7.sol


## 2단계 : 컴파일

In [80]:
!solc src/MathMultiply7.sol --combined-json abi,bin > src/MathMultiply7.json

In [81]:
!type C:\Users\15Z970-G.AA5BK\Code\jsr\src\MathMultiply7.json

{"contracts":{"src/MathMultiply7.sol:Math":{"abi":"[{\"inputs\":[{\"internalType\":\"uint256\",\"name\":\"amount\",\"type\":\"uint256\"}],\"name\":\"deposit\",\"outputs\":[],\"stateMutability\":\"payable\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"getAddressOfM7\",\"outputs\":[{\"internalType\":\"address\",\"name\":\"\",\"type\":\"address\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"getBalanceOfM7\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"getBalanceOfThis\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"multiply\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"send11M7\",\"outputs\":[],\"stateMutability\":\"p

## 3단계 : 배포

var _abiArray=JSON.parse(_json.contracts["src/MathMultiply.sol:Math"].abi);

에서 []안에 들어갈 내용은 컴파일 시 contracts 다음에 나오는 부분을 복붙해서 넣으면된다.

In [85]:
%%writefile src/MathMultiply7DeployFromFile.js

var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var fs=require('fs'); //파일시스템으로 읽는다. 
var _str = fs.readFileSync("src/MathMultiply7.json");
var _json=JSON.parse(_str)
var _abiArray=JSON.parse(_json.contracts["src/MathMultiply7.sol:Math"].abi);
//console.log(_abiArray);
//var _bin=_json.contracts.sHello2.bin;
var _bin="0x"+_json.contracts["src/MathMultiply7.sol:Math"].bin;
//console.log(_bin);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray) //await로 컨트랙이 생성될 때 해시코드랑 주소를 받음
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/MathMultiply7DeployFromFile.js


In [86]:
!node src/MathMultiply7DeployFromFile.js

Deploying the contract from 0xC44e483669dD1609bA680B47bA3aE59ccA5404b4
hash: 0x30a2c5df43332527369c13c1c0168c270e6aae8c4912dc237f5cc3a7192f7a6b
---> The contract deployed to: 0xdb3a271D6b9E3C5429C91E2eF1C70FC10aB24b1B


## 4단계 : 사용

In [87]:
%%writefile src/MathMultiply7UseFromFile.js

var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/MathMultiply7.json");
var _json=JSON.parse(_str)
var _abiArray=JSON.parse(_json.contracts["src/MathMultiply7.sol:Math"].abi);
var _instance = new web3.eth.Contract(_abiArray, "0xdb3a271D6b9E3C5429C91E2eF1C70FC10aB24b1B");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    _instance.methods.multiply().call().then(console.log); //multiply함수 불러서 실행결과 보고, 
    _instance.methods.deposit(123).send({from:accounts[0], value:123}); //deposit하고
    _instance.methods.getBalanceOfM7().call().then(console.log); //잔고 확인하고 
    
    await _instance.methods.send11M7().send({from:accounts[0]}); //fallback함수 호출되는지 보고
    _instance.methods.getBalanceOfM7().call().then(console.log); 
    //잔고까지 확인함. 근데 주소 확인하는 건 안넣었어요. 복잡해질거같아서. 혼자 할 수 있을듯요
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()

Writing src/MathMultiply7UseFromFile.js


In [90]:
!node src/MathMultiply7UseFromFile.js

Account: 0xC44e483669dD1609bA680B47bA3aE59ccA5404b4
Balance before: 99984619959999999754
56
11
22
Balance after: 99983597999999999631
Balance diff: 1021960000012288


잔고는 0. 이거는 multiply7의 잔고임.
11 전송 후 잔고가 11로 증가한 것을 확인할 수 있음. 

In [91]:
!node src/MathMultiply7UseFromFile.js

Account: 0xC44e483669dD1609bA680B47bA3aE59ccA5404b4
Balance before: 99983597999999999631
56
22
33
Balance after: 99982576039999999508
Balance diff: 1021959999995904


# 3번 영상

### 제어 - 반복문
블록체인에서의 프로그램은 명령문 마다 비용이 발생한다는 특성이있다.
gas가 반복마다 소비되고, (반복 횟수가 증가하게 되면 gas비용이 급증하게 된다.) 한도 gasLimit을 초과하게 되면, 실행이 중지될 수 밖에 없다.


- for문
> function getSum() public returns(uint) {
  uint sum=0;
  for(uint i=0;i<10;i++) //i라는 변수를 중간에 선언할 수 있음
      sum+=i;
  return sum;
}

- while문
> function getSum() public returns(uint) {
  uint sum=0;
  uint i=0;
  do {
      sum+=i;
      i++;
  } while(i<10);
  return sum;
}

프로그램을 작성해서 gas비용을 비교해보자.

또 sum변수를 local에 두는 경우와 state variable에 두는 경우로 나누어서 비교해보자.

⇒ 결론, 변수를 local에 둔 경우, 반복횟수가 적게으면 gas비용도 작다. 근데 10000이상 반복하게 되면 리믹스가 죽고 다시 실행되는 경우도 있다.

⇒ state variable로 sum을 두면 local에 둘 때보다 더 많은 gas 비용을 쓰게된다. 여기서 10000이상 반복하게 되면 아예 gasLimit을 초과하게 된다.

In [92]:
%%writefile src/ControlFlowTest.sol

pragma solidity ^0.6.0;

contract ControlFlowTest {
    uint sum;  //최종 sum
    constructor() public {
        sum = 0;
    }
    function doForLoop() public { //지역변수로 쓰겠다 
        //sum = 0; 
        uint sumLocal = 0; //update가 계속 일어남. 이는 메모리변수를 쓰는 것.
        for(uint i = 0; i < 1001; i++) { 
            sumLocal += 1;
        }
        sum = sumLocal; //최종 sum, 최종적으로 state variable을 1번 update하고있음
    } 
    
    function doForLoop2() public {  //전역변수로 쓰겠다
        sum = 0; //초기화, 메모리변수에 쓰고있음
        //uint sumLocal = 0;
        for(uint i = 0; i < 1001; i++) {
            //sumLocal += i;
            sum += 1; //for문내에서 update를 계속 하고있음 
        }
        //sum = sumLocal;
    }
    function getSum() view public returns(uint) { //합계를 조회하는 함수
        return sum;
    }
}

Writing src/ControlFlowTest.sol


### 리믹스로 프로그램을 작성해보고, 컴파일, deploy 후 log를 통해 gas값을 확인해보자. 
### 왜 비용차이가 발생하는 것일까?
- doForLoop() - local 에서는 sum=sumLocal에서 최종적으로 1번만 state variable을 update한다.

- 반면에 doForLoop2() - state variable 에서는 sum +=1;에서 for문이 반복하는 갯수만큼 계속 update가 이루어진다.


### 제어 - 조건문
switch and goto를 제외하고, 다른 언어에서 제공하는 if, else, while, do, for, break, continue, return과 비교해서 큰 차이가 없음.

- 그냥 if(condition) ~ else 를 쓰기도 하고
- ternary operator (삼항연산자)
  : betAmount=betAmount>amount ? amount : betAmount;
  
  
직접 인사하는 컨트랙을 만들어보자.

## 실습 : 인사하는 컨트랙
"Hello"를 출력하는 프로그램을 수정해서 if문을 사용해 보자. modifier를 사용하면 조건문을 사용하지 않을 수 있다. 조건문에서 비교문을 사용하게 되는데, string은 참조타입이라 비교를 하면 주소를 비교하게 된다. 어떻게 해야 하는지 생각해보자.

리믹스로 코드를 작성하고, Deploy를 하자. 그럼 Deployed Contracts가 나타난다.
- 코드를 작성할 때 Deploy할 때 값을 넘겨주기로 했으니 Deploy에 "Hello"라고 넣어주고 deploy를 다시 눌러주자.
- 그러고나면 sayHello에 Hello가 들어가 있다. 
- getBalance는 0이 나올 것이고
- compareTo에 "Hello"를 하면 결과가 true가 나오고,
- compareTo에 "hello"를 하면 결과로 false가 나온다.
- setHello를 누르고나면, Owner가 되고, sayHello를 누르면 Hello로 변한것을 확인할 수 있다. 

## 1단계 : 컨트랙 생성

In [1]:
%%writefile src/Hello1.sol

pragma solidity ^0.6.0;

contract Hello1 {
    string private hello; // 가시성은 private로 하고, 아무도 못보게 한다
    address private owner; // 주소 owner도 하나 만든다
    event PrintLog(address addr, string s); // 주소와 문자열을 받는 이벤트를 하나 만든다
    
    constructor(string memory _hello) public { // 오버로딩해서 string이니 memory 참조타입
        hello = _hello; // state variable을 초기화
        owner = msg.sender; // msg.sender는 사용자측에서 컨트랙으로 보내는 트랜잭션을 말함. (msg를 블록체인에 보내는 사람의 주소를 말함)
    }
    
    function sayHello() view public returns(string memory) { //인사하는 문자열이니 메모리로 
        return hello;
    }
    
    modifier isOwner() { //isOwner를 쓰게되면, 이 경우만 특정 함수가 실행될 수 있게끔 하여 if문이 줄어들 수 있음
        if (msg.sender != owner) {
            revert(); // exception이 발생해서 중단하는 것
        }
        _; //continue executing rest of method body, // 위의 것을 호출하는게 _;이거임
    }
    
    function setHello() public { // state variable은 가급적 사용하지 말아야함
        string memory s = "";// local로 문자열을 하나 만들고, 최종적으로 state variable을 수정하는 걸로 해야함
        if (msg.sender == owner) { //Owner일 때 Hello라고 인사하는 문자열
            s = "Hello"; 
            emit PrintLog(msg.sender, s); // 이벤트를 발생시킴(주소, 문자열)
        } else {
            s = "Olleh"; //Owner가 아닐 때 Olleh라고 인사함
            emit PrintLog(msg.sender, s); // 이벤트를 발생시킴(주소, 문자열)
        }
        hello = s; // 여기서 최종적으로 state variable을 1번 update함(update 수는 비용에 영향을 미침)
    }
    
    function compareTo(string memory _str) view public returns(bool) { // 문자열을 받아서 hello와 비교하는 함수, 결과는 boolean으로 함
        bool isEqual = false; //우선 false를 넣고, 같으면 true로 대체한다.
        //참조타입이 아니면  if (hello == _str) {
            // String(이 외에 array나 mapping도)은reference 타입이기에, 해싱을 해서 진짜 문자열을 비교해야함.
        if (keccak256(bytes(hello)) == keccak256(bytes(_str))) { // 해시문자열을 비교
            isEqual = true;
        }
        return isEqual; // if문이 끝나고 반환함
    }
    
    function getBalance() view public isOwner returns(uint) { // 오직 Owner만 볼 수 있는 잔고
        return address(this).balance;
    } //Owner일 때는 balance를 보고, Owner가 아닐 때에는 balance를 보지 못하게 코딩할 수 있다는 것 잊지말기!! 
}

Overwriting src/Hello1.sol


## 2단계 : 컴파일

보통 방식으로 컴파일하는 것.
(위에서 한 것은 파일형태로 저장해서 컴파일 한 것이다.)

In [2]:
!solc --abi --bin --gas src/Hello1.sol


======= src/Hello1.sol:Hello1 =======
Gas estimation:
construction:
   infinite + 349800 = infinite
external:
   compareTo(string):	infinite
   getBalance():	1063
   sayHello():	infinite
   setHello():	infinite
Binary:
608060405234801561001057600080fd5b506040516108d53803806108d58339818101604052602081101561003357600080fd5b810190808051604051939291908464010000000082111561005357600080fd5b8382019150602082018581111561006957600080fd5b825186600182028301116401000000008211171561008657600080fd5b8083526020830192505050908051906020019080838360005b838110156100ba57808201518184015260208101905061009f565b50505050905090810190601f1680156100e75780820380516001836020036101000a031916815260200191505b50604052505050806000908051906020019061010492919061014c565b5033600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550506101f1565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f1061018d57805160ff1916838001

위에서 보면 지금
> Gas estimation:
construction:
   infinite + 349800 = infinite
external:
   compareTo(string):	infinite
   getBalance():	1063
   sayHello():	infinite
   setHello():	infinite
 
이렇게 나오고있다. 물론 걱정은 해야하지만, 자신이 생각하기에 코드가 간단하면 infinite가 나오는걸 굳이 신경쓰지는 않아도 됨

사실 이렇게 나오면 리믹스에서 디버그로 gas값을 확인해봐야함. 뭐.. 그러렴

## 3단계 : 배포

In [3]:
%%writefile src/Hello1Deploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
//var _abiArray = JSON.parse(_abiStr);
var _abiArray = [{"inputs":[{"internalType":"string","name":"_hello","type":"string"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"addr","type":"address"},{"indexed":false,"internalType":"string","name":"s","type":"string"}],"name":"PrintLog","type":"event"},{"inputs":[{"internalType":"string","name":"_str","type":"string"}],"name":"compareTo","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"sayHello","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"setHello","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _bin = "0x" + "608060405234801561001057600080fd5b506040516108d53803806108d58339818101604052602081101561003357600080fd5b810190808051604051939291908464010000000082111561005357600080fd5b8382019150602082018581111561006957600080fd5b825186600182028301116401000000008211171561008657600080fd5b8083526020830192505050908051906020019080838360005b838110156100ba57808201518184015260208101905061009f565b50505050905090810190601f1680156100e75780820380516001836020036101000a031916815260200191505b50604052505050806000908051906020019061010492919061014c565b5033600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550506101f1565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f1061018d57805160ff19168380011785556101bb565b828001600101855582156101bb579182015b828111156101ba57825182559160200191906001019061019f565b5b5090506101c891906101cc565b5090565b6101ee91905b808211156101ea5760008160009055506001016101d2565b5090565b90565b6106d5806102006000396000f3fe608060405234801561001057600080fd5b506004361061004c5760003560e01c806312065fe01461005157806327906b251461006f57806373fde99a14610142578063ef5fb05b1461014c575b600080fd5b6100596101cf565b6040518082815260200191505060405180910390f35b6101286004803603602081101561008557600080fd5b81019080803590602001906401000000008111156100a257600080fd5b8201836020820111156100b457600080fd5b803590602001918460018302840111640100000000831117156100d657600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001838380828437600081840152601f19601f820116905080830192505050505050509192919290505050610231565b604051808215151515815260200191505060405180910390f35b61014a6102be565b005b610154610558565b6040518080602001828103825283818151815260200191508051906020019080838360005b83811015610194578082015181840152602081019050610179565b50505050905090810190601f1680156101c15780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b6000600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161461022b57600080fd5b47905090565b6000806000905082805190602001206000604051808280546001816001161561010002031660029004801561029d5780601f1061027b57610100808354040283529182019161029d565b820191906000526020600020905b815481529060010190602001808311610289575b5050915050604051809103902014156102b557600190505b80915050919050565b6060604051806020016040528060008152509050600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161415610435576040518060400160405280600581526020017f48656c6c6f00000000000000000000000000000000000000000000000000000081525090507fd2065d4a97074e981ab7a6ff96587a468761d4c023459e930236e2f49ef6f64f3382604051808373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200180602001828103825283818151815260200191508051906020019080838360005b838110156103f55780820151818401526020810190506103da565b50505050905090810190601f1680156104225780820380516001836020036101000a031916815260200191505b50935050505060405180910390a161053e565b6040518060400160405280600581526020017f4f6c6c656800000000000000000000000000000000000000000000000000000081525090507fd2065d4a97074e981ab7a6ff96587a468761d4c023459e930236e2f49ef6f64f3382604051808373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200180602001828103825283818151815260200191508051906020019080838360005b838110156105025780820151818401526020810190506104e7565b50505050905090810190601f16801561052f5780820380516001836020036101000a031916815260200191505b50935050505060405180910390a15b80600090805190602001906105549291906105fa565b5050565b606060008054600181600116156101000203166002900480601f0160208091040260200160405190810160405280929190818152602001828054600181600116156101000203166002900480156105f05780601f106105c5576101008083540402835291602001916105f0565b820191906000526020600020905b8154815290600101906020018083116105d357829003601f168201915b5050505050905090565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f1061063b57805160ff1916838001178555610669565b82800160010185558215610669579182015b8281111561066857825182559160200191906001019061064d565b5b509050610676919061067a565b5090565b61069c91905b80821115610698576000816000905550600101610680565b5090565b9056fea264697066735822122040533632f933d92bba30a1b190de0a9ae27aafc3107eb93f088989c0b788414c64736f6c63430006030033";
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin, arguments: ["Hello from web3"]})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/Hello1Deploy.js


In [4]:
!node src/Hello1Deploy.js

Deploying the contract from 0xF017348B65a03c82e40A6802e5fD27733A8Adf42
hash: 0x9cd8c8b2afe1d34a05132ab5ffdf175d77b52cfd78daf7fd725070cfbbf5e7da
---> The contract deployed to: 0x6cBFfbe9c1c87BEc797Cd7b1067af189b628FFBC


## 4단계 : 사용

In [5]:
%%writefile src/Hello1Use.js

var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var _abiArray = [{"inputs":[{"internalType":"string","name":"_hello","type":"string"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"addr","type":"address"},{"indexed":false,"internalType":"string","name":"s","type":"string"}],"name":"PrintLog","type":"event"},{"inputs":[{"internalType":"string","name":"_str","type":"string"}],"name":"compareTo","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"sayHello","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"setHello","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var hello = new web3.eth.Contract(_abiArray, "0x6cBFfbe9c1c87BEc797Cd7b1067af189b628FFBC");
var event = hello.events.PrintLog(function (error, result) {
    if (!error) { //이벤트가 있으면 발생시킴
        console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    hello.methods.sayHello().call().then(console.log);  //null
    
    await hello.methods.setHello().send({from: accounts[0]});
    hello.methods.sayHello().call().then(console.log);
    hello.methods.compareTo("Hello").call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    
}
doIt()

Overwriting src/Hello1Use.js


In [6]:
!node src/Hello1Use.js

Account: 0xF017348B65a03c82e40A6802e5fD27733A8Adf42
Balance before: 99990384160000000000
Hello from web3
Hello
true
Balance after: 99989755300000000000
Balance diff: 628859999993856


--------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------

## 블록체인에서 컨트랙 삭제 
원래 프로그램에서 오류가 발생하거나 쓸모가 없어지면 유지보수나 삭제를 해야한다.

그러나 블록체인은 유지보수(수정)가 불가능하다. 그래서 오류가 발생하거나 쓸모없어진 컨트랙은 삭제를 해야한다.

컨트랙은 Read Only로서, 만들어지면 영원히 남는다. 

selfdestruct()는 잔고를 전액 반환하고, 컨트랙을 블록체인으로부터 제거한다. 

리믹스에서 만들어보자. (Hello2.sol)

## 1단계 : 컨트랙 생성

In [1]:
%%writefile src/Hello2.sol

pragma solidity ^0.6.0;

contract Hello2 {
    string hello; //Hello 문자열 선언
    address payable owner; //Owner 선언

    event PrintLog(string _s);
    constructor() public {
        owner = msg.sender;
    }
    function sayHello() view public returns(string memory) {
        return hello;
    }
    function setHello(string memory _hello) public payable {
        hello = _hello;
        emit PrintLog(_hello);
    }
    function getBalance() view public returns(uint) {
        return address(this).balance;
    }
    //Balance를 반환하기 때문에 kill()을 이용하면 잔고 확인이 불가하다. 왜냐하면 컨트랙이 블록체인에서 제거되었기 때문.
    function kill() public { // 따라서 kill()하기 전에 잔고 확인 함수를 만들고, 그 잔고만큼 web3에서 coinbase에 그 잔액이 들어왔는지를 확인해야 함
        if (msg.sender == owner)  //지우는 권한은 오직 owner에게만 주어져야 한다. 
        selfdestruct(owner); //owner를 넣어서 지움
        
    }
    
}

Overwriting src/Hello2.sol


## 2단계 : 컴파일

In [2]:
!solc src/Hello2.sol --combined-json abi,bin > src/Hello2.json

In [3]:
!type C:\Users\15Z970-G.AA5BK\Code\jsr\src\Hello2.json

{"contracts":{"src/Hello2.sol:Hello2":{"abi":"[{\"inputs\":[],\"stateMutability\":\"nonpayable\",\"type\":\"constructor\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"internalType\":\"string\",\"name\":\"_s\",\"type\":\"string\"}],\"name\":\"PrintLog\",\"type\":\"event\"},{\"inputs\":[],\"name\":\"getBalance\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"kill\",\"outputs\":[],\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"sayHello\",\"outputs\":[{\"internalType\":\"string\",\"name\":\"\",\"type\":\"string\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[{\"internalType\":\"string\",\"name\":\"_hello\",\"type\":\"string\"}],\"name\":\"setHello\",\"outputs\":[],\"stateMutability\":\"payable\",\"type\":\"function\"}]","bin":"608060405234801561001057600080fd5b5033600160006101000a81548173ffffffffffffffffffffffffff

## 3단계 : 배포
컴파일 후 abi, bin을 저장한 파일에서 읽어서 배포하자.

In [4]:
%%writefile src/Hello2DeployFromFile.js

var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/Hello2.json");
var _json=JSON.parse(_str)
//var _abiArray=JSON.parse(_json.contracts.sHello2.abi);
var _abiArray=JSON.parse(_json.contracts['src/Hello2.sol:Hello2'].abi);
//var _bin=_json.contracts.sHello2.bin;
var _bin="0x"+_json.contracts['src/Hello2.sol:Hello2'].bin;

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/Hello2DeployFromFile.js


In [5]:
!node src/Hello2DeployFromFile.js

Deploying the contract from 0xe5579beB6314d4530A60106Cc57289FeEFd00632
hash: 0xe488369b40f9eb7f5874f626f8ac00da348b942eb35c14ea4dabf8682a4bec7a
---> The contract deployed to: 0x5FC347F39E48CdB6033e941f053e422b34355708


## 4단계 : 사용

In [1]:
%%writefile src/Hello2UseFromFile.js

var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/Hello2.json");
var _json=JSON.parse(_str)
//var _abiArray=JSON.parse(_json.contracts.sHello2.abi);
var _abiArray=JSON.parse(_json.contracts['src/Hello2.sol:Hello2'].abi);

var hello = new web3.eth.Contract(_abiArray, "0x5FC347F39E48CdB6033e941f053e422b34355708");
var event = hello.events.PrintLog(function (error, result) {
    if (!error) {
        console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    hello.methods.sayHello().call().then(console.log);  //null
    await hello.methods.setHello("Hello World!").send({from: accounts[0], value: 1111});
    hello.methods.sayHello().call().then(console.log);
    //hello.methods.getBalance().call(function(err, bal) { console.log("Contract Balance: "+bal) });
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    //hello.methods.kill().send({from: accounts[0]}) 
    //명시적으로 kill 함수를 불러보자. 처음에 kill() 주석을 해제하지 않고 node를 실행하면 결과가 보이지만, 
    //kill()을 하고 나서 다시 node로 실행해보면 오류가 뜰 것이다.
}

doIt()

Overwriting src/Hello2UseFromFile.js


In [7]:
!node src/Hello2UseFromFile.js

Account: 0xe5579beB6314d4530A60106Cc57289FeEFd00632
Balance before: 99993390460000000000

Hello World!
Balance after: 99992485659999998889
Balance diff: 904800000000000


kill을 하고나서 재호출하면 Error : Returned values aren't valid, did it run Out of Gas?가 호출된다는데 나는 안나옴... 저 gas가 고갈되었다는 거는 gas를 너무 많이 써서 그런 경우도 있지만 아예 실행이 되지 않을때에도 나오는 오류임.